In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#read files

df1 = pd.read_csv("data/positions_mock1.csv")
incert1 = pd.read_csv("data/positions_mock1_sigma_rel_pos.csv")
tca1 = '2024-05-24 19:39:41.572 +0100'

df2 = pd.read_csv("data/positions_mock2.csv")
incert2 = pd.read_csv("data/positions_mock2_sigma_rel_pos.csv")
tca2 = '2024-05-23 11:33:31.053 +0100'

df3 = pd.read_csv("data/positions_mock3.csv")
incert3 = pd.read_csv("data/positions_mock3_sigma_rel_pos.csv")
tca3 = '2024-05-25 00:43:03.001 +0100'

In [5]:
df1 = df1.drop(columns=['relative_position_n', 'relative_position_r', 'relative_position_t'])
df2 = df2.drop(columns=['relative_position_n', 'relative_position_r', 'relative_position_t'])
df3 = df3.drop(columns=['relative_position_n', 'relative_position_r', 'relative_position_t'])

In [6]:
#add incert1 to df1 
df1['relative_position_sigma_r'] = incert1['relative_position_sigma_r']
df1['relative_position_sigma_n'] = incert1['relative_position_sigma_n']
df1['relative_position_sigma_t'] = incert1['relative_position_sigma_t']

#add incert2 to df2
df2['relative_position_sigma_r'] = incert2['relative_position_sigma_r']
df2['relative_position_sigma_n'] = incert2['relative_position_sigma_n']
df2['relative_position_sigma_t'] = incert2['relative_position_sigma_t']

#add incert3 to df3
df3['relative_position_sigma_r'] = incert3['relative_position_sigma_r']
df3['relative_position_sigma_n'] = incert3['relative_position_sigma_n']
df3['relative_position_sigma_t'] = incert3['relative_position_sigma_t']

In [9]:
def calculate_mean(df, column_name):
    group = df.groupby('creation_date')[column_name].mean().tolist()

    if np.isnan(group).all():
        group = []
    
    return group

def add_bounds(df, dataset, column_name):
    if column_name == 'AlongTrack':
        letter = 'n'
    elif column_name == 'CrossTrack':
        letter = 't'
    elif column_name == 'Radial':
        letter = 'r'
    dataset['upper_bound'][column_name] = list(dataset['dt'][column_name] + df['relative_position_sigma_' + letter])
    dataset['lower_bound'][column_name] = list(dataset['dt'][column_name] - df['relative_position_sigma_' + letter])

    return dataset

def make_dataset(df, tca):     
    tag = df['tags'].unique()

    datasets = {}

    for t in tag:
        split1 = t.split(',')
        split = split1[1].split('_')
        dt_time = list(df[df['tags'] == t]['creation_date'].unique())

        if 'target' and 'chaser' in split:
            n = split[0] + ' vs ' + split[0]
            target = split[0]
            chaser = split[0]
            color = '#FFD521'
        elif 'target' in split:
            n = split[0] + ' vs HAC'
            target = split[0]
            chaser = 'HAC'
            color = '#00ACC1'
        elif 'chaser' in split:
            n = 'HAC vs ' + split[0]
            target = 'HAC'
            chaser = split[0]
            color = '#DDEA1F'
        elif 'baseline' in split:
            n = 'HAC vs HAC'
            target = 'HAC'
            chaser = 'HAC'
            color = '#A1173C'
        else:
            print('Error')
          
        datasets[n] = {
            'tca': tca, 'name': n, 'target': target, 'chaser': chaser, 'dt_time': dt_time, 'color': color,
            'dt': {
                'MissDistance': calculate_mean(df[df['tags'] == t], 'miss_distance'),
                'AlongTrack': list(abs(np.array(df[df['tags'] == t]['target_along_track'].unique()) - np.array(df[df['tags'] == t]['chaser_along_track'].unique()))), 
                'CrossTrack': list(abs(np.array(df[df['tags'] == t]['target_cross_track'].unique()) - np.array(df[df['tags'] == t]['chaser_cross_track'].unique()))), 
                'Radial': list(abs(np.array(df[df['tags'] == t]['target_radial'].unique()) - np.array(df[df['tags'] == t]['chaser_radial'].unique())))
            },
            'upper_bound': {
                'AlongTrack': [], 
                'CrossTrack': [], 
                'Radial': []
            },
            'lower_bound': {
                'AlongTrack': [], 
                'CrossTrack': [], 
                'Radial': []
            },
            'Total':{
                'alert': 500,
                'warning': 1500
            },
            'Radial':{
                'alert': 50,
                'warning': 100
            },

        }

        datasets[n] = add_bounds(df[df['tags'] == t], datasets[n], 'AlongTrack')
        datasets[n] = add_bounds(df[df['tags'] == t], datasets[n], 'CrossTrack')
        datasets[n] = add_bounds(df[df['tags'] == t], datasets[n], 'Radial')

    return datasets


In [10]:
import json

with open('data/data1.json', 'w') as f:
    json.dump(make_dataset(df1, tca1), f)
with open('data/data2.json', 'w') as f:
    json.dump(make_dataset(df2, tca2), f)
with open('data/data3.json', 'w') as f:
    json.dump(make_dataset(df3, tca3), f)